### Install enoslib (+ small setup)

In [5]:
#!pip install -U pip
#!pip install -y enoslib
#!pip install python-grid5000
ingi = True
if ingi:
    !PYTHONPATH=/etinfo/users/2020/adgiot/.local/lib/python3.10/site-packages/

In [ ]:
# need for credentials in .env
#!echo -e "username: $(head -n 1 .env)\npassword: $(head -n 2 .env | tail -n 1)" > ~/.python-grid5000.yaml

In [6]:
import enoslib as en
from grid5000 import Grid5000
from pathlib import Path

conf_file = Path.home() / ".python-grid5000.yaml"
gk = Grid5000.from_yaml(conf_file)
gk.sites.list() # if credentials were written adequatly in .env, this should work

[<Site uid:grenoble>,
 <Site uid:lille>,
 <Site uid:luxembourg>,
 <Site uid:lyon>,
 <Site uid:nancy>,
 <Site uid:nantes>,
 <Site uid:rennes>,
 <Site uid:sophia>,
 <Site uid:toulouse>]

### Launch machines

In [9]:
import enoslib as en
from grid5000 import Grid5000
from pathlib import Path

conf_file = Path.home() / ".python-grid5000.yaml"
gk = Grid5000.from_yaml(conf_file)
gk.sites.list() # if credentials were written adequatly in .env, this should work

job_name = 'dev'

conf = (
    en.G5kConf.from_settings(
        job_name=job_name,
        job_type=[],#"deploy"],
        key="/home/adrien/.ssh/grid",
        #env_name="ubuntu2204-min",
        walltime="3:00:00"
    )    
    #.add_machine(roles=["client"], cluster="econome", nodes=1)
    #.add_machine(roles=["server"], cluster="grimoire", nodes=2)
    .add_machine(roles=["client"], cluster="grimoire", nodes=1)
    .add_machine(roles=["server"], cluster="grimoire", nodes=1)
)

provider = en.G5k(conf)

roles, networks = provider.init()

Output()

Finished 1 tasks (Granting root access on the nodes (sudo-g5k)) on 
{'grimoire-3.nancy.grid5000.fr', 'grimoire-4.nancy.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

In [5]:
provider.jobs

[<Job uid:4016744 site:nancy state:running user:agiot>]

### Setup the machines

In [46]:
results = en.run_command("cd /home/agiot/infinity; make library; make examples", roles=roles["server"])
for result in results:
    #print(f"{result.host} has {result.payload['stdout']} logical CPU cores")
    print(result.payload['stdout'])

Output()

Finished 1 tasks (cd /home/agiot/infinity; make library; make examples) on 
{'grimoire-4.nancy.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

mkdir -p release
ar rvs release/libinfinity.a build/infinity/core/Context.o build/infinity/memory/Atomic.o build/infinity/memory/Buffer.o build/infinity/memory/Region.o build/infinity/memory/RegionToken.o build/infinity/memory/RegisteredMemory.o build/infinity/queues/QueuePair.o build/infinity/queues/QueuePairFactory.o build/infinity/requests/RequestToken.o build/infinity/utils/Address.o
r - build/infinity/core/Context.o
r - build/infinity/memory/Atomic.o
r - build/infinity/memory/Buffer.o
r - build/infinity/memory/Region.o
r - build/infinity/memory/RegionToken.o
r - build/infinity/memory/RegisteredMemory.o
r - build/infinity/queues/QueuePair.o
r - build/infinity/queues/QueuePairFactory.o
r - build/infinity/requests/RequestToken.o
r - build/infinity/utils/Address.o
rm -rf release/include
cp --parents src/infinity/infinity.h src/infinity/core/Context.h src/infinity/core/Configuration.h src/infinity/memory/Atomic.h src/infinity/memory/Buffer.h src/infinity/memory/Region.h src/infinity/me

In [47]:
results = en.run_command("cd /home/agiot/tcp-echo; g++ -O3 -pthread -o client.o client.cpp; g++ -O3 -pthread -o server.o server.cpp; ", roles=roles["server"])
for result in results:
    #print(f"{result.host} has {result.payload['stdout']} logical CPU cores")
    print(result.payload['stdout'])

Output()

Finished 1 tasks (cd /home/agiot/tcp-echo; g++ -O3 -pthread -o client.o client.cpp; g++ -O3 
-pthread -o server.o server.cpp; ) on {'grimoire-4.nancy.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

### Run the experiments

In [21]:
import time

results_s = en.run_command("cd /home/agiot/tcp-echo; ./server.o", roles=roles["server"], background=True)
results_c = en.run_command("cd /home/agiot/tcp-echo; ./client.o", roles=roles["client"])

for result in results_c:
    if 'failed' in result.stdout:
        print('socket aint killed yet')
        time.sleep(5)
    print(result.stdout)


Output()

Finished 1 tasks (cd /home/agiot/tcp-echo; ./server.o) on {'grimoire-4.nancy.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

Output()

Finished 1 tasks (cd /home/agiot/tcp-echo; ./client.o) on {'grimoire-3.nancy.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

Socket successfully created..
connected to the server..
Socket successfully created..
Socket successfully binded..
Server listening..
server accept the client...
it's fine
397
261
195
261
303
281
167
206
305
197
215
178
175
284
279
243
280
280
155
225
283
169
162
161
211
133
313
228
243
252
298
241
249
309
284
284
274
264
169
231
207
271
300
184
226
304
221
258
284
232
164
186
271
241
257
306
232
247
291
217
306
289
301
234
185
263
304
261
275
195
207
195
217
197
203
185
188
182
316
236
255
120
211
120
300
231
313
272
302
207
181
310
193
304
240
167
205
281
222
192
202
161
185
163
265
270
281
236
177
203
262
237
263
305
259
274
272
246
191
302
293
127
158
293
259
164
184
116
257
166
298
271
184
192
198
154
260
297
276
270
286
207
92
268
259
265
261
195
125
162
295
122
206
199
170
205
255
229
246
246
229
157
171
298
174
286
254
299
287
196
303
239
285
265
159
171
249
204
147
149
151
156
292
266
297
268
158
154
173
253
259
300
246
257
192
197
313
206
224
262
210
177
259
284
172
195
147
2

In [21]:
result.stdout.split('\n')[7:]

['372',
 '320',
 '313',
 '255',
 '320',
 '267',
 '342',
 '131',
 '222',
 '190',
 '264',
 '245',
 '251',
 '315',
 '227',
 '341',
 '273',
 '262',
 '296',
 '326',
 '349',
 '256',
 '318',
 '341',
 '274',
 '298',
 '320',
 '227',
 '298',
 '310',
 '284',
 '250',
 '350',
 '339',
 '288',
 '338',
 '235',
 '232',
 '268',
 '335',
 '247',
 '345',
 '267',
 '327',
 '324',
 '337',
 '329',
 '319',
 '310',
 '318',
 '308',
 '288',
 '325',
 '243',
 '256',
 '319',
 '238',
 '286',
 '255',
 '322',
 '250',
 '314',
 '335',
 '237',
 '330',
 '258',
 '235',
 '212',
 '239',
 '267',
 '309',
 '292',
 '321',
 '257',
 '337',
 '313',
 '338',
 '305',
 '243',
 '253',
 '350',
 '322',
 '334',
 '253',
 '321',
 '333',
 '295',
 '255',
 '252',
 '174',
 '301',
 '344',
 '298',
 '255',
 '319',
 '325',
 '286',
 '313',
 '319',
 '315',
 '318',
 '344',
 '275',
 '322',
 '341',
 '262',
 '340',
 '330',
 '326',
 '271',
 '360',
 '236',
 '237',
 '297',
 '343',
 '286',
 '339',
 '325',
 '256',
 '242',
 '318',
 '296',
 '335',
 '314',
 '343',


In [15]:
#_ = en.run_command("cd /home/agiot/infinity; ./release/examples/read-write-send -s", roles=roles["server"], background=True)
#results_c = en.run_command("cd /home/agiot/infinity; ./release/examples/read-write-send", roles=roles["client"])
for result in results_c:
    print(result.stdout.split('\n')[4:])

['118', '42', '37', '35', '43', '34', '35', '34', '39', '51', '37', '37', '35', '35', '35', '34', '39', '35', '36', '35', '33', '9', '12', '35', '36', '35', '35', '33', '36', '35', '36', '34', '42', '35', '33', '33', '66', '34', '35', '34', '34', '33', '34', '17', '36', '34', '34', '35', '34', '36', '41', '34', '34', '33', '32', '33', '33', '12', '33', '33', '35', '42', '33', '34', '20', '34', '34', '33', '32', '49', '32', '13', '33', '34', '33', '33', '33', '32', '57', '34', '34', '33', '32', '33', '32', '33', '35', '34', '32', '31', '33', '31', '34', '34', '34', '33', '32', '31', '33', '38', '10', '33', '32', '32', '34', '33', '35', '33', '32', '32', '31', '31', '32', '32', '32', '31', '32', '31', '30', '31', '32', '31', '31', '31', '30', '30', '31', '31', '40', '32', '32', '32', '31', '32', '32', '31', '33', '31', '32', '31', '47', '31', '31', '31', '31', '31', '32', '17', '31', '31', '32', '32', '31', '31', '14', '32', '32', '31', '30', '31', '32', '15', '31', '32', '32', '30', '31

In [48]:
rdma_echo_times = []
tcp_echo_times = []
for _ in range(3):
    _ = en.run_command("cd /home/agiot/infinity; ./release/examples/read-write-send -s", roles=roles["server"], background=True)
    results_c = en.run_command("cd /home/agiot/infinity; ./release/examples/read-write-send", roles=roles["client"])
    for result in results_c:
        cur_echo_times = result.stdout.split('\n')[4:]
        rdma_echo_times.append(cur_echo_times)
    
    finished = False
    while not finished:
        _ = en.run_command("cd /home/agiot/tcp-echo; ./server.o", roles=roles["server"], background=True)
        results_c = en.run_command("cd /home/agiot/tcp-echo; ./client.o", roles=roles["client"])

        for result in results_c:
            if 'failed' in result.stdout:
                print('socket aint killed yet')
                time.sleep(5)
            else:
                cur_echo_times = result.stdout.split('\n')[7:]
                tcp_echo_times.append(cur_echo_times)
                finished = True
    
rdma_echo_times
tcp_echo_times

Output()

Finished 1 tasks (cd /home/agiot/infinity; ./release/examples/read-write-send -s) on 
{'grimoire-4.nancy.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

Output()

Finished 1 tasks (cd /home/agiot/infinity; ./release/examples/read-write-send) on 
{'grimoire-3.nancy.grid5000.fr'}

...

─────────────────────────────────────────────────────────────────────────────────────────────

Output()

Finished 1 tasks (cd /home/agiot/tcp-echo; ./client.o) on {'grimoire-3.nancy.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

[['413',
  '340',
  '350',
  '344',
  '345',
  '337',
  '298',
  '335',
  '396',
  '408',
  '482',
  '543',
  '615',
  '645',
  '706',
  '774',
  '770',
  '2583',
  '2030',
  '1528',
  '1538',
  '1582',
  '2867',
  '2311',
  '1773',
  '1804',
  '1833',
  '3061',
  '2505',
  '2042',
  '1963',
  '2021',
  '3304',
  '2757',
  '2216',
  '2197',
  '2245',
  '3545',
  '2993',
  '2459',
  '2478',
  '2479',
  '3782',
  '3227',
  '2739',
  '2749',
  '2766',
  '3994',
  '3440',
  '2923',
  '2897',
  '2962',
  '4180',
  '3623',
  '3091',
  '3169',
  '3217',
  '4457',
  '3899',
  '3403',
  '3423',
  '3412',
  '4707',
  '4144',
  '3625',
  '3580',
  '3627',
  '4880',
  '4322',
  '3791',
  '3801',
  '3894',
  '5092',
  '4534',
  '4060',
  '3999',
  '4059',
  '5331',
  '4766',
  '4245',
  '4212',
  '4262',
  '5514',
  '4962',
  '4430',
  '4464',
  '4500',
  '5737',
  '5181',
  '4671',
  '4667',
  '4730',
  '6046',
  '5489',
  '4979',
  '4990',
  '5020',
  '6286',
  '5734',
  '5219',
  '5176',
  '5224

In [49]:
import pandas as pd  

to_save = {'rdma1':rdma_echo_times[0],
           'rdma2':rdma_echo_times[1],
           'rdma3':rdma_echo_times[2],
           'tcp1':tcp_echo_times[0],
           'tcp2':tcp_echo_times[1],
          'tcp3':tcp_echo_times[2]}

df = pd.DataFrame(to_save) 
    
# saving the dataframe 
df.to_csv('res_usleep_700.csv')

In [50]:
len(tcp_echo_times[0])


1000

### Kill every last one of them

In [8]:
# Release all Grid'5000 resources
provider.destroy()